In [1]:
%matplotlib inline

In [2]:
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import is_aa, three_to_one
import scipy.spatial
import pandas as pd
import numpy as np
import glob
import shutil

In [3]:
def process_residue(residue):
    '''
    Processes a single residue to determine the coordinates of the alpha-carbon
    and the sidechain center-of-mass. Also checks for missing atoms in a
    residue.
    (This is directly from Ben's code with slight modifications for Glycine)--AJH
    '''
    output_dict = {}
    atoms_seen = []
    # Convert three letter amino acid to one letter
    try:
        output_dict['amino_acid'] = three_to_one(residue.resname)
    except KeyError:
        return 'Error'
    
    # Grab residue number AND any insertion site labeling (11A, 11B, etc.)
    output_dict['residue'] = str(residue.get_id()[1]) + \
                             residue.get_id()[2].strip()
    output_dict['chain'] = residue.get_full_id()[2]
    try:
        int(output_dict['residue'])
    except:
        return 'Error'
    
    # Coordinates of all sidechain atoms in this residue
    sidechain_coords = []
    for atom in residue:
        atoms_seen.append(atom.name)
        if atom.name == 'CA':
            # Save alpha-carbon coordinates
            output_dict['CA'] = atom.get_coord()
            
        ######################################################
        #AJH: code to add CB and to make exceptions for Glycine
            if residue.resname=='GLY':
                output_dict['CB'] = atom.get_coord()
                
        if atom.name == 'CB':
            # Save beta-carbon coordinates
            output_dict['CB'] = atom.get_coord()
        ######################################################

        if atom.name not in ['C', 'CA', 'O', 'N']:
            # Must be a sidechain atom...
            sidechain_coords.append(atom.get_coord())
            
    if 'CA' not in output_dict or 'CB' not in output_dict:
        return 'Error'
    
    for mainchain_atom in ['N', 'C', 'O']:
        # Warn about any missing mainchain atoms
        if mainchain_atom not in atoms_seen:
            return 'Error'

    if len(sidechain_coords) == 0:
        if output_dict['amino_acid'] == 'G':
            sidechain_coords.append(output_dict['CA'])
        else:
            return 'Error'
    # Calculate side chain center of mass
    output_dict['SCcenter'] = sum(sidechain_coords)/\
                                      len(sidechain_coords)
    
    return output_dict


def process_residue_complete(residue):
    '''
    Thisis a temporary test function
    '''
    output_dict = {}
    atoms_seen = []
    # Convert three letter amino acid to one letter
    try:
        output_dict['amino_acid'] = three_to_one(residue.resname)
    except KeyError:
        return 'Error'
    
    # Grab residue number AND any insertion site labeling (11A, 11B, etc.)
    output_dict['residue'] = str(residue.get_id()[1]) + \
                             residue.get_id()[2].strip()
    output_dict['chain'] = residue.get_full_id()[2]
    try:
        int(output_dict['residue'])
    except:
        return 'Error'
    
    # Coordinates of all atoms in this residue
    all_atom_coords = []
    sidechain_coords = []
    for atom in residue:
        atoms_seen.append(atom.name)
        if output_dict['amino_acid'] == 'G':
            if atom.name == 'CA':
                sidechain_coords.append(atom.get_coord())
        
        all_atom_coords.append(atom.get_coord())
        
        if atom.name not in ['C', 'CA', 'O', 'N']:
            # Must be a sidechain atom...
            sidechain_coords.append(atom.get_coord())

    # Calculate side chain center of mass
    output_dict['all_atoms'] = all_atom_coords
    output_dict['sc_atoms'] = sidechain_coords
    
    return output_dict


def unit_vector(vector):
    ''' Returns the unit vector of the vector.'''
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    '''
    Returns the angle in radians between vectors 'v1' and 'v2':
    
            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    '''

    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

# Writing distance matrices identified by different amino acid reference points

The initial input should be a "cleaned" PDB file of the type that is used directly for simulations, which is to say properly numbered 1...len(amino acid sequence), no random (het) atoms, single chain, etc.

In [4]:
error_pdbs = [] ### Keep track of pdb's where weird things happened to investigate
for infile_loc in glob.glob('../Data/PDBs/*.pdb')[:]: #Iterating through a bunch of files
    print("###################################################")
    print(infile_loc)
    print("###################################################")
    prot_name = infile_loc.split('/')[-1].replace('.rosetta.pdb', '')
    print(prot_name)
    structure = PDBParser().get_structure(prot_name, infile_loc)

    ###Get residue coordinates
    residues = []
    for residue in structure.get_residues():
        if is_aa(residue):
            temp = process_residue(residue)
            residues.append(temp)
            
    if 'Error' in residues:
        error_pdbs.append(infile_loc)
        continue
    residues = sorted(residues, key=lambda x: int(x['residue']))
    
    ###Get the overall amino acid sequence. Know there are much more efficient ways than this.
    aa_seq_from_structure = sorted([(i['amino_acid'], int(i['residue'])) for i in residues], key=lambda x: x[1])
    aa_seq_from_structure = ''.join([i[0] for i in aa_seq_from_structure])
    print('Protein length is {} amino acids'.format(len(aa_seq_from_structure)))
    print(aa_seq_from_structure)
    ###Write the amino acid sequence to a fasta
    with open(infile_loc.replace('PDBs/', 'fastas/').replace('.pdb', '.fasta'), 'w') as outfile:
        outfile.write('>{}\n'.format(prot_name))
        outfile.write('{}'.format(aa_seq_from_structure))
    
    
    ###On to contact matrices
    for calculation in ['CA', 'SCcenter', 'CB']: #Writing three different contact matrices
        print(calculation)
        numbering = list(range(1, len(aa_seq_from_structure)+1))
        df_contacts = pd.DataFrame(index=numbering, columns=numbering) ###Set up empty df for contact dists
        for i, residue in enumerate(residues):
            for j, other_residue in enumerate(residues):
                contact_dist = scipy.spatial.distance.euclidean(residue[calculation],other_residue[calculation])
                df_contacts.set_value(i+1, j+1, contact_dist)
        contact_file = infile_loc.replace('.pdb', '_{}_contacts.csv'.format(calculation))
        df_contacts.to_csv(contact_file)
        
        ###Separate code to get a matrix of side chain angles
        if calculation == 'CA':
            print('Working on angles')
            numbering = list(range(1, len(aa_seq_from_structure)+1))
            df_angles = pd.DataFrame(index=numbering, columns=numbering)
            for i, residue in enumerate(residues):
                for j, other_residue in enumerate(residues):
                    if residue['amino_acid'] != 'G' and i != j:
                        point1 = np.array(other_residue['CA'])
                        point2 = np.array(residue['SCcenter'])
                        vertex = np.array(residue['CA'])
                        v1 = point1 - vertex
                        v2 = point2 - vertex
                        angle = angle_between(v1, v2)
                    else:
                        angle = np.nan
                    df_angles.set_value(i+1, j+1, angle)
            angles_file = infile_loc.replace('/PDBs/', '/Contact_matrices/').replace('.rosetta', '')
            angles_file = angles_file.replace('.pdb', '_{}_angles.csv'.format(calculation))
            df_angles.to_csv(angles_file)

###################################################
../Data/PDBs/1FCY_A.pdb
###################################################
1FCY_A.pdb
Protein length is 236 amino acids
ASPQLEELITKVSKAHQETFPSLCQLGKYTTNSSADHRVQLDLGLWDKFSELATKCIIKIVEFAKRLPGFTGLSIADQITLLKAACLDILMLRICTRYTPEQDTMTFSDGLTLNRTQMHNAGFGPLTDLVFAFAGQLLPLEMDDTETGLLSAICLICGDRMDLEEPEKVDKLQEPLLEALRLYARRRRPSQPYMFPRMLMKITDLRGISTKGAERAITLKMEIPGPMPPLIREMLE
CA
Working on angles
SCcenter
CB


In [ ]:
print(error_pdbs)

In [ ]:
for pdb_file in error_pdbs:
    print(pdb_file)
    shutil.move(pdb_file, pdb_file.replace('/PDBs/', '/PDBs/unknown_errors/'))

# Writing distance matrices identified by minimum atomic distance

In [5]:
error_pdbs_v2 = []
for infile_loc in glob.glob('../Data/PDBs/*rosetta.pdb')[:]: #Iterating through a bunch of files
    print("###################################################")
    print(infile_loc)
    print("###################################################")
    prot_name = infile_loc.split('/')[-1].replace('.rosetta.pdb', '')
    print(prot_name)
    structure = PDBParser().get_structure(prot_name, infile_loc)

    ###Get residue coordinates
    residues = []
    for residue in structure.get_residues():
        if is_aa(residue):
            temp = process_residue_complete(residue)
            residues.append(temp)
            
    if 'Error' in residues:
        error_pdbs_v2.append(infile_loc)
        continue
    residues = sorted(residues, key=lambda x: int(x['residue']))
    
    ###Get the overall amino acid sequence. Know there are much more efficient ways than this.
    aa_seq_from_structure = sorted([(i['amino_acid'], int(i['residue'])) for i in residues], key=lambda x: x[1])
    aa_seq_from_structure = ''.join([i[0] for i in aa_seq_from_structure])
    print('Protein length is {} amino acids'.format(len(aa_seq_from_structure)))
    print(aa_seq_from_structure)
    
    for calculation in ['all_atoms', 'sc_atoms']: #Writing three different contact matrices
        print(calculation)
        numbering = list(range(1, len(aa_seq_from_structure)+1))
        df_contacts = pd.DataFrame(index=numbering, columns=numbering) ###Set up empty df for contact dists
        for i, residue in enumerate(residues):
            for j, other_residue in enumerate(residues):
                contact_dist = np.min(scipy.spatial.distance.cdist(\
                                residue[calculation], other_residue[calculation], 'euclidean'))   
                df_contacts.set_value(i+1, j+1, contact_dist)
        contact_file = infile_loc.replace('/PDBs/', '/Contact_matrices/').replace('.rosetta', '')
        contact_file = contact_file.replace('.pdb', '_{}_contacts.csv'.format(calculation+'_min'))
        df_contacts.to_csv(contact_file)

###################################################
../Data/PDBs/1EK0_A.rosetta.pdb
###################################################
1EK0_A
Protein length is 168 amino acids
VTSIKLVLLGEAAVGKSSIVLRFVSNDFAENKEPTIGAAFLTQRVTINEHTVKFEIWDTAGQERFASLAPYYRNAQAALVVYDVTKPQSFIKARHWVKELHEQASKDIIIALVGNKIDLQEGGERKVAREEGEKLAEEKGLLFFETSAKTGENVNDVFLGIGEKIPLK
all_atoms
sc_atoms
